In [1]:
from model import *
from config import *
from data_processing import *
today = datetime.date.today()

In [2]:
config = TsConf()
df = TsDataFrame(pd.read_csv(config.cmapss_url))
df.columns = config.columns_old
df = df[config.columns_new]

In [ ]:
df = df.cluster(config)

In [ ]:
cols = config.groupids + [config.timestamp] + config.opset + config.select_feat + ["RUL", "CLUSTER"]

In [ ]:
df.loc[df.FILEID==102, cols].to_csv("NASA_102_unnormalized.csv", index=False)

In [ ]:
df = df.data_norm(config)

In [ ]:
config.ewm = 20
df = df.ewma(config)

In [ ]:
df.loc[df.FILEID==102, cols].to_csv("NASA_102_normalized.csv", index=False)

In [ ]:
train = df[(["FILEID","ENGINEID", "TIMECYCLE"]+config.select_feat+["RUL"])][df.FILEID.isin(config.train_fids)]
test = df[(["FILEID","ENGINEID", "TIMECYCLE"]+config.select_feat+["RUL"])][df.FILEID.isin(config.test_fids)]

In [ ]:
train.tail()

# Model training

## LSTM (Sibur)

In [ ]:
n_eng_train = max(train.ENGINEID)
n_eng_test = max(test.ENGINEID)

In [ ]:
train.target = config.target
train.groupids = config.groupids
train.timestamp = config.timestamp
test.target = config.target
test.groupids = config.groupids
test.timestamp = config.timestamp

In [ ]:
Train = False
Predict = not Train

In [ ]:
config.sequence_length = 1
config.batch_size = 1
config.shift = 1
config.restore_model = True
config.save_model = True
config.random = False
config.stateful = True
config.train_fids = [102]
config.test_fids = [202]

In [ ]:
config.display()

In [ ]:
m = TsLSTM(config)

In [ ]:
if Train:
    m.train(train, test, config)

In [ ]:
if Predict:
    dtrain = train #.loc[train.ENGINEID<9]
    y_target = dtrain["RUL"]
    y_pred = m.predict(dtrain, config)
    z=pd.DataFrame(dict(target=y_target, pred=y_pred)).reset_index()
    #plt.plot(z[["target", "pred"]])

In [ ]:
len(df[df.FILEID==102])

In [ ]:
len(z)

In [ ]:
z[["pred","target"]].to_csv("NASA_102_predictions.csv", index=False)

In [ ]:
#pd.DataFrame(out)

In [ ]:
#y_submit = full_prediction*std_y+mean_y
#sub_file = pd.DataFrame()
#sub_file["timestamp"] = submit_X.index[-2872:]
#sub_file["target"] = y_submit[-2872:]
#sub_file.to_csv('submit2.csv', index=False)

## Gradient Boosting Regression

### TS fresh dataset

In [ ]:
#add_lag_roll(test.iloc[:,:-2], feat_names, lag=lag, step=step, roll=0).head()

In [ ]:
#df.groupby(["FILEID", "ENGINEID"])

In [ ]:
#X=pd.DataFrame()
#y=list()

In [ ]:
#for name, group in df.groupby(["FILEID", "ENGINEID"]):
    #X = pd.concat([X, add_lag_roll(group[feat_names], feat_names, lag=lag, step=step)], axis=0)
    #y = y.append(group["RUL"][lag:])
    
    

In [ ]:
#X.tail(1).values